In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import datetime
import pandas_profiling
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
unan_comments = pd.read_csv('toxicity_annotated_comments_unanimous.tsv', sep='\t')
unan_scores = pd.read_csv('toxicity_annotations_unanimous.tsv', sep='\t')

# remove newline and tab tokens
unan_comments['comment'] = unan_comments['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
unan_comments['comment'] = unan_comments['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))

In [ ]:
comment_score = pd.merge(unan_comments, unan_scores, on='rev_id')
split = comment_score.groupby('toxicity')

toxic_group = split.get_group(1).groupby('comment')
nontoxic_group = split.get_group(0).groupby('comment')

toxics = []
nontoxics = []
all_comments = comment_score['comment']

for name, group in toxic_group:
    toxics.append(name)

for name, group in nontoxic_group:
    nontoxics.append(name)

In [ ]:
# create the transform
toxic_vectorizer = TfidfVectorizer()
nontoxic_vectorizer = TfidfVectorizer()
all_vectorizer = TfidfVectorizer()
# tokenize and build vocab
toxic_vectorizer.fit(toxics)
nontoxic_vectorizer.fit(nontoxics)
all_vectorizer.fit(all_comments)

In [ ]:
# summarize
print(all_vectorizer.vocabulary_)
print("-------------------------")
print(all_vectorizer.idf_)

toxic_vocab_words = toxic_vectorizer.vocabulary_

In [ ]:
toxic_vocab = pd.Series(toxic_vectorizer.vocabulary_)
toxic_vocab1 = pd.Series(toxic_vocab.index.values, index=toxic_vocab )
toxic_vocab_tfidf = pd.Series(toxic_vectorizer.idf_)
toxic_vocab_score = pd.concat([toxic_vocab1, toxic_vocab_tfidf], keys=['Words','Scores'], axis=1)

nontoxic_vocab = pd.Series(nontoxic_vectorizer.vocabulary_)
nontoxic_vocab1 = pd.Series(nontoxic_vocab.index.values, index=nontoxic_vocab )
nontoxic_vocab_tfidf = pd.Series(nontoxic_vectorizer.idf_)
nontoxic_vocab_score = pd.concat([nontoxic_vocab1, nontoxic_vocab_tfidf], keys=['Words','Scores'], axis=1)

all_vocab = pd.Series(all_vectorizer.vocabulary_)
all_vocab1 = pd.Series(all_vocab.index.values, index=all_vocab )
all_vocab_tfidf = pd.Series(all_vectorizer.idf_)
all_vocab_score = pd.concat([all_vocab1, all_vocab_tfidf], keys=['Words','Scores'], axis=1)

merged = pd.merge(nontoxic_vocab_score, toxic_vocab_score, on="Words")
merged = pd.merge(merged, all_vocab_score, on="Words")

In [ ]:
score_diff_non_toxic = []
score_diff_non_all = []
score_diff_toxic_all = []


for index, row in merged.iterrows():
    score_diff_non_toxic.append(row['Scores_x'] - row['Scores_y'])
    score_diff_toxic_all.append(row['Scores_y'] - row['Scores'])
    score_diff_non_all.append(row['Scores_x'] - row['Scores'])
    
merged['Score Diff, Toxic and Non-Toxic'] = score_diff_non_toxic
merged['Score Diff, All and Non-Toxic'] = score_diff_non_all
merged['Score Diff, All and Toxic'] = score_diff_toxic_all